# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.124973e+02     1.721779e+00
 * time: 0.12695002555847168
     1     1.106770e+01     7.840131e-01
 * time: 1.3303358554840088
     2    -1.159227e+01     9.450275e-01
 * time: 1.4795889854431152
     3    -3.373532e+01     7.207410e-01
 * time: 1.6194438934326172
     4    -4.733251e+01     5.947558e-01
 * time: 1.7752039432525635
     5    -5.689242e+01     2.374373e-01
 * time: 1.9240970611572266
     6    -5.977812e+01     2.097292e-01
 * time: 2.0403048992156982
     7    -6.091933e+01     8.463507e-02
 * time: 2.1494719982147217
     8    -6.141150e+01     3.787424e-02
 * time: 2.271008014678955
     9    -6.169164e+01     3.843542e-02
 * time: 2.3801119327545166
    10    -6.186685e+01     2.884645e-02
 * time: 2.496212959289551
    11    -6.198345e+01     2.614652e-02
 * time: 2.612082004547119
    12    -6.206882e+01     1.545582e-02
 * time: 2.720974922180176
    13    -6.212725e+01     1.315479e-02
 * time: 2.8385589122772

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671075
    AtomicLocal         -18.8557752
    AtomicNonlocal      14.8522700
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485390 
    Xc                  -19.3336827

    total               -62.261666456170
